# Module 06: Income Tracking & Malaysian Tax Compliance

**Difficulty**: ⭐

**Estimated Time**: 45 minutes

**Prerequisites**: [Module 03: Winning Proposals](03_winning_proposals.ipynb)

## Learning Objectives

By the end of this notebook, you will be able to:
1. Track all income and expenses systematically
2. Understand Malaysian tax requirements at your income level
3. Know when to register for SSM and tax filing
4. Calculate actual take-home after all fees
5. Generate monthly income reports

## Setup

In [ ]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import json

# Malaysian tax constants
TAX_THRESHOLD_RM = 37333  # Annual income below this = no tax
SSM_REGISTRATION_COST_RM = 30  # Annual business registration
USD_TO_MYR = 4.3

def init_income_tracker():
    """Initialize income tracking CSV."""
    tracker_file = Path('../data/tracking/income.csv')
    tracker_file.parent.mkdir(parents=True, exist_ok=True)
    
    if not tracker_file.exists():
        df = pd.DataFrame(columns=[
            'date', 'client_name', 'project_description', 
            'gross_usd', 'platform_fee_usd', 'net_usd',
            'net_rm', 'payment_method', 'payment_fee_rm', 'final_rm'
        ])
        df.to_csv(tracker_file, index=False)
        print(f"✓ Created income tracker: {tracker_file}")
    else:
        print(f"✓ Income tracker exists: {tracker_file}")
    
    return tracker_file

income_tracker_path = init_income_tracker()
print("✓ Income tracking tools loaded")

## Part 1: Understanding Malaysian Tax Requirements

### Your Tax Situation at RM690/month

In [ ]:
def assess_tax_requirements(annual_income_rm):
    """
    Determine tax obligations based on annual income.
    
    Args:
        annual_income_rm: Total income for the year in RM
    
    Returns:
        Dictionary with tax requirements and recommendations
    """
    tax_required = annual_income_rm >= TAX_THRESHOLD_RM
    percentage_of_threshold = (annual_income_rm / TAX_THRESHOLD_RM) * 100
    
    # Estimate tax (simplified)
    if annual_income_rm < TAX_THRESHOLD_RM:
        estimated_tax = 0
    elif annual_income_rm < 50000:
        # First RM5,000 after threshold is taxed at 1%
        taxable = annual_income_rm - TAX_THRESHOLD_RM
        estimated_tax = min(taxable, 5000) * 0.01
    else:
        # Progressive rates apply
        estimated_tax = "Consult tax advisor"
    
    # Recommendations
    if annual_income_rm < 10000:
        level = "Very Low Income"
        actions = [
            "✓ No tax registration required",
            "✓ No tax filing required",
            "→ Keep simple income records",
            "→ Consider SSM registration for professionalism (RM30/year)",
        ]
    elif annual_income_rm < 34000:
        level = "Below Threshold"
        actions = [
            "✓ No tax filing required (yet)",
            "→ Keep detailed income/expense records",
            "→ Strongly consider SSM registration",
            "→ Track expenses for future deductions",
        ]
    elif annual_income_rm < 50000:
        level = "Tax Filing Required"
        actions = [
            "✓ Must file tax return (Form BE)",
            "✓ Register with LHDN",
            "✓ Mandatory SSM registration",
            "→ Claim business expense deductions",
            f"→ Estimated tax: RM{estimated_tax:.2f}" if isinstance(estimated_tax, float) else "→ Consult tax advisor",
        ]
    else:
        level = "Higher Income - Get Professional Help"
        actions = [
            "✓ Mandatory tax filing",
            "✓ Consider hiring accountant",
            "✓ Quarterly estimated payments may apply",
        ]
    
    return {
        'annual_income_rm': annual_income_rm,
        'tax_threshold_rm': TAX_THRESHOLD_RM,
        'percentage_of_threshold': percentage_of_threshold,
        'tax_required': tax_required,
        'estimated_tax': estimated_tax,
        'level': level,
        'actions': actions,
    }

# Your target income
monthly_target = 690
annual_target = monthly_target * 12

# Different scenarios
scenarios = [
    {'label': 'Current Target', 'annual_rm': annual_target},
    {'label': '2x Target', 'annual_rm': annual_target * 2},
    {'label': 'Near Threshold', 'annual_rm': 35000},
    {'label': 'Above Threshold', 'annual_rm': 45000},
]

print("MALAYSIAN TAX REQUIREMENTS BY INCOME LEVEL")
print("="*70)
print(f"Tax Threshold: RM{TAX_THRESHOLD_RM:,}/year\n")

for scenario in scenarios:
    result = assess_tax_requirements(scenario['annual_rm'])
    
    print(f"\n{scenario['label']}: RM{result['annual_income_rm']:,}/year")
    print(f"  Status: {result['level']}")
    print(f"  % of threshold: {result['percentage_of_threshold']:.0f}%")
    print(f"  Tax required: {'Yes' if result['tax_required'] else 'No'}")
    if isinstance(result['estimated_tax'], float):
        print(f"  Estimated tax: RM{result['estimated_tax']:.2f}")
    print(f"  Action items:")
    for action in result['actions']:
        print(f"    {action}")

print("\n" + "="*70)
print(f"\nAT YOUR TARGET (RM{monthly_target}/month = RM{annual_target}/year):")
print("You are at {:.0f}% of tax threshold".format((annual_target/TAX_THRESHOLD_RM)*100))
print("✓ NO TAX FILING REQUIRED")
print("✓ NO TAX OWED")
print("✓ Keep simple records just in case")

### Exercise 1: Calculate Your Tax Position

**Task**: Based on your projected income for the year, determine your tax requirements.

In [ ]:
# YOUR projected income - update these monthly!
my_income_projection = {
    'Month 1': 345,   # RM
    'Month 2': 620,
    'Month 3': 970,
    'Month 4': 1240,
    'Month 5': 1380,
    'Month 6': 1610,
    'Month 7-12': 1500,  # Average for remaining months
}

# Calculate annual total
first_6_months = sum(list(my_income_projection.values())[:6])
remaining_6_months = my_income_projection['Month 7-12'] * 6
annual_total_rm = first_6_months + remaining_6_months

print("YOUR INCOME PROJECTION")
print("="*70)
print("\nFirst 6 months:")
cumulative = 0
for month, income in list(my_income_projection.items())[:6]:
    cumulative += income
    print(f"  {month}: RM{income:.0f} (Cumulative: RM{cumulative:.0f})")

print(f"\nMonths 7-12: RM{my_income_projection['Month 7-12']:.0f} average/month")
print(f"\nProjected Annual Total: RM{annual_total_rm:,.0f}")

# Assess tax position
tax_assessment = assess_tax_requirements(annual_total_rm)

print("\n" + "="*70)
print("\nYOUR TAX POSITION:")
print(f"  Income level: {tax_assessment['level']}")
print(f"  Tax filing required: {'Yes' if tax_assessment['tax_required'] else 'No'}")
print(f"  Estimated tax: RM{tax_assessment['estimated_tax']}" if isinstance(tax_assessment['estimated_tax'], float) else f"  Estimated tax: {tax_assessment['estimated_tax']}")
print(f"\n  Recommendations:")
for action in tax_assessment['actions']:
    print(f"    {action}")

## Part 2: Income Tracking System

### Record Every Payment

In [ ]:
def record_income(client_name, project_desc, gross_usd, platform='upwork', payment_method='wise'):
    """
    Record an income transaction with all fee calculations.
    """
    # Platform fees
    platform_fees = {'upwork': 0.10, 'fiverr': 0.20, 'direct': 0.0}
    platform_fee_usd = gross_usd * platform_fees.get(platform, 0.10)
    net_usd = gross_usd - platform_fee_usd
    
    # Convert to RM
    net_rm = net_usd * USD_TO_MYR
    
    # Payment processing fees
    if payment_method == 'wise':
        payment_fee_rm = (net_rm * 0.005) + 1.06  # 0.5% + RM1.06
    elif payment_method == 'payoneer':
        payment_fee_rm = net_rm * 0.005  # ~0.5%
    else:  # paypal
        payment_fee_rm = (net_rm * 0.084) + 50  # Much higher!
    
    final_rm = net_rm - payment_fee_rm
    
    # Record to CSV
    tracker_file = Path('../data/tracking/income.csv')
    df = pd.read_csv(tracker_file)
    
    new_record = {
        'date': datetime.now().strftime('%Y-%m-%d'),
        'client_name': client_name,
        'project_description': project_desc,
        'gross_usd': gross_usd,
        'platform_fee_usd': platform_fee_usd,
        'net_usd': net_usd,
        'net_rm': net_rm,
        'payment_method': payment_method,
        'payment_fee_rm': payment_fee_rm,
        'final_rm': final_rm,
    }
    
    df = pd.concat([df, pd.DataFrame([new_record])], ignore_index=True)
    df.to_csv(tracker_file, index=False)
    
    return {
        'record_number': len(df),
        'gross_usd': gross_usd,
        'final_rm': final_rm,
        'total_fees': (gross_usd * USD_TO_MYR) - final_rm,
        'retention_pct': (final_rm / (gross_usd * USD_TO_MYR)) * 100,
    }

# Example: Record some income
print("RECORDING INCOME EXAMPLES")
print("="*70)

result1 = record_income(
    client_name="Sarah M.",
    project_desc="Data cleaning - customer database",
    gross_usd=75,
    platform='upwork',
    payment_method='wise'
)

print(f"\nIncome Record #{result1['record_number']}:")
print(f"  Gross: ${result1['gross_usd']} USD")
print(f"  Final: RM{result1['final_rm']:.2f}")
print(f"  Fees: RM{result1['total_fees']:.2f}")
print(f"  Retained: {result1['retention_pct']:.1f}%")

result2 = record_income(
    client_name="John K.",
    project_desc="Web scraper for products",
    gross_usd=100,
    platform='upwork',
    payment_method='wise'
)

print(f"\nIncome Record #{result2['record_number']}:")
print(f"  Gross: ${result2['gross_usd']} USD")
print(f"  Final: RM{result2['final_rm']:.2f}")
print(f"  Retained: {result2['retention_pct']:.1f}%")

print("\n" + "="*70)
print("✓ Income records saved to: ../data/tracking/income.csv")

### Generate Monthly Reports

In [ ]:
def generate_monthly_report(year=None, month=None):
    """
    Generate income report for specific month or overall.
    """
    tracker_file = Path('../data/tracking/income.csv')
    
    if not tracker_file.exists():
        return "No income recorded yet."
    
    df = pd.read_csv(tracker_file)
    
    if len(df) == 0:
        return "No income recorded yet."
    
    # Convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Filter by month if specified
    if year and month:
        df = df[(df['date'].dt.year == year) & (df['date'].dt.month == month)]
        period = f"{year}-{month:02d}"
    else:
        period = "All Time"
    
    if len(df) == 0:
        return f"No income for {period}"
    
    # Calculate totals
    total_projects = len(df)
    total_gross_usd = df['gross_usd'].sum()
    total_final_rm = df['final_rm'].sum()
    total_fees_rm = (df['gross_usd'].sum() * USD_TO_MYR) - total_final_rm
    avg_project_rm = total_final_rm / total_projects
    
    report = f"""
INCOME REPORT: {period}
{'='*70}

Summary:
  Projects completed: {total_projects}
  Gross income: ${total_gross_usd:.2f} USD
  Total fees: RM{total_fees_rm:.2f}
  Net income: RM{total_final_rm:.2f}
  Average per project: RM{avg_project_rm:.2f}

Projects:
"""
    
    for idx, row in df.iterrows():
        report += f"  {row['date'].strftime('%Y-%m-%d')}: {row['client_name']} - ${row['gross_usd']:.0f} → RM{row['final_rm']:.2f}\n"
    
    # Tax assessment
    # Estimate annual based on this month (if monthly report)
    if year and month:
        estimated_annual = total_final_rm * 12
        report += f"\nProjected Annual (if sustained): RM{estimated_annual:.2f}\n"
        if estimated_annual < TAX_THRESHOLD_RM:
            report += "✓ Still below tax threshold\n"
        else:
            report += "⚠ Would exceed tax threshold - tax filing required\n"
    
    return report

# Generate report
print(generate_monthly_report())

# You can also generate for specific month:
# print(generate_monthly_report(year=2025, month=11))

## Part 3: SSM Registration Guide

### When and How to Register

In [ ]:
ssm_registration_guide = {
    'Why Register': [
        '✓ Professional credibility with clients',
        '✓ Ability to claim business expense deductions',
        '✓ Required if income exceeds RM50,000/year',
        '✓ Costs only RM30/year (very affordable)',
        '✓ Online process takes under 1 hour',
    ],
    'When to Register': [
        'Optional: At any income level for professionalism',
        'Recommended: Once earning RM1,500+/month consistently',
        'Mandatory: If annual income exceeds RM50,000',
    ],
    'Registration Steps': [
        '1. Go to ezbiz.ssm.com.my',
        '2. Click "New Registration" → "Enterprise"',
        '3. Choose "Sole Proprietor" (for individual)',
        '4. Enter business name (can use your own name)',
        '5. Describe business: "Freelance IT Services" or "Data Analysis Services"',
        '6. Pay RM30 via online banking',
        '7. Receive certificate within 1 hour (PDF)',
    ],
    'What You Need': [
        'MyKad (Malaysian IC)',
        'Business name (can be "[Your Name] Freelance Services")',
        'RM30 for annual registration',
        'Online banking access for payment',
    ],
    'After Registration': [
        'Renew annually (RM30/year)',
        'Keep certificate safe (PDF and print)',
        'Include registration number on invoices (optional but professional)',
        'Track business expenses for tax deductions',
    ],
}

print("SSM REGISTRATION GUIDE")
print("="*70)

for section, points in ssm_registration_guide.items():
    print(f"\n{section.upper()}:")
    for point in points:
        print(f"  {point}")

print("\n" + "="*70)
print("\nRECOMMENDATION FOR RM690/MONTH TARGET:")
print("  Registration is OPTIONAL but recommended for professionalism.")
print("  Consider registering after your 3rd or 4th successful project.")
print("  RM30/year is negligible (less than 1 hour of work at $10/hr).")

## Part 4: Deductible Business Expenses

### Track These for Future Tax Deductions

In [ ]:
deductible_expenses = {
    'Internet & Phone': {
        'examples': 'Monthly internet bill (RM150-200), phone plan',
        'deductible': '100% if used primarily for business, or portion used',
        'record': 'Keep monthly bills',
    },
    'Computer & Equipment': {
        'examples': 'Laptop, monitor, keyboard, mouse, headset',
        'deductible': 'Full cost if purchased for business use',
        'record': 'Keep purchase receipts',
    },
    'Software & Subscriptions': {
        'examples': 'GitHub Pro, Claude Pro, hosting, domain names',
        'deductible': '100%',
        'record': 'Keep subscription receipts/invoices',
    },
    'Office Supplies': {
        'examples': 'Notebooks, pens, printer paper, ink',
        'deductible': '100%',
        'record': 'Keep purchase receipts',
    },
    'Professional Development': {
        'examples': 'Online courses (Udemy, Coursera), books, certifications',
        'deductible': '100% if work-related',
        'record': 'Keep receipts and course completion certificates',
    },
    'Home Office (Partial)': {
        'examples': 'Portion of rent/utilities for dedicated workspace',
        'deductible': 'Percentage based on space used (e.g., 10% of rent)',
        'record': 'Calculate percentage, keep rental agreement',
    },
}

print("DEDUCTIBLE BUSINESS EXPENSES")
print("="*70)
print("\nIMPORTANT: Keep receipts even if you're below tax threshold.")
print("These become valuable once your income grows.\n")

for category, details in deductible_expenses.items():
    print(f"\n{category}:")
    print(f"  Examples: {details['examples']}")
    print(f"  Deductible: {details['deductible']}")
    print(f"  Record keeping: {details['record']}")

# Calculate potential savings
typical_annual_expenses = {
    'Internet': 1800,  # RM150/month
    'Software': 500,   # Various subscriptions
    'Equipment': 1000, # Laptop accessories
    'Courses': 300,    # Professional development
}

total_expenses = sum(typical_annual_expenses.values())

print("\n" + "="*70)
print("\nTYPICAL ANNUAL BUSINESS EXPENSES:")
for category, amount in typical_annual_expenses.items():
    print(f"  {category}: RM{amount}")
print(f"\nTotal: RM{total_expenses}")
print(f"\nIf your income reaches RM40,000/year (taxable):")
print(f"  These RM{total_expenses} in deductions could save ~RM50-100 in taxes.")

## Summary

### What You've Learned

1. ✓ Malaysian tax requirements at different income levels
2. ✓ How to track income systematically
3. ✓ When to register with SSM
4. ✓ Deductible business expenses
5. ✓ Monthly reporting and projections

### Key Takeaways

- **At RM690/month**: No tax filing required (only 22% of threshold)
- **Record everything**: Even when not required, it's good practice
- **SSM registration**: Optional but recommended (RM30/year)
- **Track expenses**: Internet, software, equipment all deductible
- **Foreign-sourced income**: Tax-exempt through 2036 (Budget 2025)

### Your Action Items

1. ☐ Set up income tracking spreadsheet
2. ☐ Record every payment received
3. ☐ Keep all receipts (even when below tax threshold)
4. ☐ Generate monthly income reports
5. ☐ Consider SSM registration once earning RM1,500+/month
6. ☐ Review tax position quarterly

### When You Reach Different Milestones

**RM690-1,500/month**:
- Continue simple tracking
- No tax obligations
- Consider SSM for professionalism

**RM1,500-3,000/month**:
- Strongly recommend SSM registration
- Track all business expenses
- Still likely below tax threshold

**Above RM3,000/month** (RM36,000/year):
- Register with LHDN for tax file
- Mandatory SSM registration
- Set aside 10-15% for potential taxes
- Consider consulting tax advisor

### Additional Resources

- SSM Registration: https://ezbiz.ssm.com.my
- LHDN (Tax): https://www.hasil.gov.my
- Tax calculator: https://www.imoney.my/income-tax-calculator